In [2]:
print('31 paskaita\n(pasiruošta)')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector as cnt

# import os
# import glob

31 paskaita
(pasiruošta)


In [8]:
# Turite nusiskaityti iš SQL DB lentelės autopliuslt į Pandas DataFrame 
# gamintoją, kainą, pagaminimo datą, ridą, pavarų tipą, variklio informaciją. 
# SQL užklausoje turite atmesti tuos įrašus, kur rida nenurodyta, 
# taip pat palikite tik nesikartojančius įrašus 
# (atranka pagal ID stuleplį - jei ID sutampa - reiškia, jog skelbimas dubliuojasi)

#prisijungimas prie mySQL, ir kursoriaus sukūrimas

DB = cnt.connect(
host = '88.223.121.130',
user='user',
password='slaptazodis',
port=3306,
database='works'
)
C=DB.cursor()

#sql užklausa per pandas sql_query
sql="""select gamintojas g, price k, rida as r, price as Kn€ from autopliuslt"""
df = pd.read_sql_query(sql, con=DB) # Sukuriamas Pandas DF.
df.head(2)

/tmp/ipykernel_58115/3210438067.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, con=DB) # Sukuriamas Pandas DF.


,g,k,r,Kn€
0,BMW,19 500,253 000 km,19 500
1,Mitsubishi,2 700,195 405 km,2 700
